In [5]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import os
from datetime import datetime
import time
import io
import re

# --- Yardımcı Fonksiyon: MultiIndex Sütunları Düzleştirme ---
def flatten_multiindex_columns(df):
    """
    Pandas DataFrame'inin çok seviyeli (MultiIndex) sütun başlıklarını düzleştirir.
    Boş veya 'Unnamed' başlıkları daha anlamlı hale getirir.
    """
    if isinstance(df.columns, pd.MultiIndex):
        flattened_columns = []
        for col_level_0, col_level_1 in df.columns:
            # Üst seviye başlık "Unnamed" veya boş ise alt seviye başlığı kullan
            if 'Unnamed' in col_level_0 or not col_level_0.strip():
                # Eğer alt seviye de "Unnamed" veya boş ise, genel bir isim atama
                # Burada, DataFrame'in ilk birkaç satırını kontrol ederek daha anlamlı isimler bulmaya çalışabiliriz
                if 'Unnamed' in col_level_1 or not col_level_1.strip():
                    # Eğer sütunun kendisi de boş veya anlamsızsa, geçici bir isim ver
                    if df.iloc[:, len(flattened_columns)].name: # Sütun adını doğrudan kontrol etmeye çalış
                         flattened_columns.append(df.iloc[:, len(flattened_columns)].name.strip())
                    else:
                        flattened_columns.append(f"Column_{len(flattened_columns) + 1}")
                else:
                    flattened_columns.append(col_level_1.strip())
            else:
                # Üst seviye başlık anlamlı ise, üst ve alt seviyeyi birleştir
                if 'Unnamed' in col_level_1 or not col_level_1.strip():
                    flattened_columns.append(col_level_0.strip())
                else:
                    flattened_columns.append(f"{col_level_0.strip()} - {col_level_1.strip()}")
        df.columns = flattened_columns
    else:
        # Zaten tek seviyeli ise sadece boşlukları temizle
        df.columns = [col.strip() for col in df.columns]
    return df

# --- Yardımcı Fonksiyon: Tek Bir Takım Sayfasındaki Tabloları Kazıma ---
def scrape_team_page_tables(team_url, team_name_safe, output_base_directory, driver):
    """
    Belirli bir takımın detay sayfasındaki tüm istatistik tablolarını çeker ve kaydeder.
    """
    print(f"\n--- '{team_name_safe}' takımının sayfasına gidiliyor: {team_url} ---")
    driver.get(team_url)
    time.sleep(7) # Takım sayfası yüklenmesi için daha uzun bekleme
    html_content_team = driver.page_source
    soup_team = BeautifulSoup(html_content_team, 'lxml')
    print(f"'{team_name_safe}' takımının HTML içeriği çekildi ve ayrıştırıldı.")

    team_folder_path = os.path.join(output_base_directory, team_name_safe)
    if not os.path.exists(team_folder_path):
        os.makedirs(team_folder_path)
        print(f"Takım klasörü '{team_folder_path}' oluşturuldu.")

    all_tables_on_team_page = soup_team.find_all('table')

    for i, table in enumerate(all_tables_on_team_page):
        table_id = table.get('id', f'unknown_table_{i}')
        table_caption_tag = table.find('caption')
        
        # Tablo için daha anlamlı bir isim oluştur
        if table_caption_tag:
            caption_text_raw = table_caption_tag.get_text(strip=True)
            table_friendly_name = re.sub(r'[^a-zA-Z0-9_ -]', '', caption_text_raw).replace(' ', '_')
        else:
            table_friendly_name = f"Table_ID_{table_id}"
        
        # İstatistik içermeyen veya gereksiz tabloları atla (genişletilmiş kontrol)
        skip_keywords = [
            "notes", "rank_key", "all_comps_team_vs_team", 
            "player_summary", "keepers_adv", "all_results", 
            "squad_transfer_history", "squad_next_match"
        ]
        # Match logs, player stats tablolarının içinde "fixtures" gibi kelimeler geçebilir,
        # ancak biz zaten spesifik olarak o id'leri hedefleyeceğimiz için bu kontrolü 
        # daha dikkatli yapmalıyız. Sadece tamamen genel veya bilgi amaçlı tabloları atlayalım.
        
        # Eğer bu bir Match Logs tablosuysa, özel adlandırma yapalım
        if "matchlogs" in table_id.lower():
            # Caption'dan veya ID'den competition ismini çekmeye çalışalım
            competition_name = "Unknown_Competition"
            if table_caption_tag:
                # "2025-2026 Galatasaray: All Competitions Match Logs" -> "All_Competitions"
                match_comp = re.search(r':\s*(.*?)\s*Match', caption_text_raw)
                if match_comp:
                    competition_name = re.sub(r'[^a-zA-Z0-9_ -]', '', match_comp.group(1)).replace(' ', '_')
            
            table_friendly_name = f"Match_Logs_{competition_name}"
            
        # Player bazlı istatistik tabloları için daha spesifik isimler
        # Galatasaray HTML'indeki ID'leri ve genel fbref kalıplarını dikkate alarak
        elif "stats_standard" in table_id.lower():
            table_friendly_name = "Player_Standard_Stats"
        elif "stats_keeper" in table_id.lower():
            table_friendly_name = "Player_Goalkeeping_Stats"
        elif "stats_shooting" in table_id.lower():
            table_friendly_name = "Player_Shooting_Stats"
        elif "stats_playing_time" in table_id.lower():
            table_friendly_name = "Player_Playing_Time_Stats"
        elif "stats_misc" in table_id.lower():
            table_friendly_name = "Player_Miscellaneous_Stats"
        elif "stats_passing" in table_id.lower():
            table_friendly_name = "Player_Passing_Stats"
        elif "stats_passing_types" in table_id.lower():
            table_friendly_name = "Player_Passing_Types_Stats"
        elif "stats_gca" in table_id.lower(): # Goal and Shot Creation
            table_friendly_name = "Player_Goal_Shot_Creation_Stats"
        elif "stats_defense" in table_id.lower():
            table_friendly_name = "Player_Defense_Stats"
        elif "stats_possession" in table_id.lower():
            table_friendly_name = "Player_Possession_Stats"
        elif "stats_misc" in table_id.lower():
            table_friendly_name = "Player_Miscellaneous_Stats"
        
        # Son bir kontrol: eğer tablo adı hala atlama anahtar kelimelerinden birini içeriyorsa atla
        if any(keyword in table_friendly_name.lower() for keyword in skip_keywords):
            print(f"  Atlanıyor: '{table_friendly_name}' (ID: {table_id}) - istatistik tablosu değil veya gereksiz.")
            continue


        print(f"  Takım için tablo okunuyor: '{table_friendly_name}' (ID: {table_id})...")
        try:
            df = pd.read_html(io.StringIO(str(table)), flavor='lxml')[0]
            df = flatten_multiindex_columns(df) # Sütun başlıklarını düzleştir
            
            current_date = datetime.now().strftime("%Y-%m-%d")
            filename = os.path.join(team_folder_path, f"{table_friendly_name}_{current_date}.csv")
            df.to_csv(filename, index=False, encoding='utf-8')
            print(f"  '{table_friendly_name}' tablosu '{filename}' olarak başarıyla kaydedildi.")
        except Exception as e:
            print(f"  Hata: Takım sayfasındaki '{table_friendly_name}' (ID: {table_id}) tablosu işlenirken sorun oluştu: {e}")
        
        time.sleep(1) # Her takım tablosu işleme arasına kısa bir gecikme


# --- Ana Kazıma İşlemini Başlatan Fonksiyon ---
def run_super_lig_scraping():
    """
    Süper Lig'deki tüm takımların detay sayfalarındaki istatistik tablolarını kazıma sürecini başlatır.
    """
    base_url = "https://fbref.com"
    super_lig_base_url = f"{base_url}/en/comps/26/Super-Lig-Stats" # Süper Lig'in ana istatistik sayfası
    output_base_directory = "scraped_fbref_team_stats" 

    # Ana çıktı klasörünü oluştur
    if not os.path.exists(output_base_directory):
        os.makedirs(output_base_directory)
        print(f"Ana klasör '{output_base_directory}' oluşturuldu.")

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("start-maximized")
    options.add_argument("disable-infobars")
    options.add_argument("--disable-extensions")
    options.add_argument("--disable-gpu")
    options.add_argument(f'user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')
    
    print("WebDriver başlatılıyor...")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    print("WebDriver başarıyla başlatıldı.")

    try:
        # --- 1. Süper Lig Ana Sayfasından Takım URL'lerini Çekme ---
        print(f"Süper Lig ana sayfasına gidiliyor: {super_lig_base_url}")
        driver.get(super_lig_base_url)
        time.sleep(5) # Sayfanın yüklenmesi için bekle
        html_content_main = driver.page_source
        soup_main = BeautifulSoup(html_content_main, 'lxml')
        print("Süper Lig ana sayfası HTML içeriği çekildi ve ayrıştırıldı.")

        team_urls = {}
        # Ana lig tablosu genellikle 'resultsYYYY-YYYY[lig_id]_overall' ID'sine sahiptir.
        # En güncel sezonun ID'sini dinamik olarak bulmaya çalışalım.
        # Sayfadaki tüm tablo wrapper'larını bulup içlerinden doğru ID'ye sahip olanı seçelim.
        
        # Bu ID, genellikle ligin genel sonuçlar tablosudur.
        # Yorumdaki gibi belirli bir sezona ait ID'yi kullanmak en güvenlisidir.
        # Eğer bu ID her zaman aynı değilse, dinamik olarak bulmak gerekir.
        # Şimdilik varsayımsal bir ID kullanalım ve manuel listeyi koruyalım
        # Eğer güncel sezonun ID'si dinamik olarak değişiyorsa, onu bulmak için regex kullanılabilir.
        
        # Önceki koda dönersek, team_list'i elle verdiğimiz için bu kısmı direkt kullanabiliriz:
        team_list_hardcoded = {
            "Galatasaray": "/en/squads/ecd11ca2/Galatasaray-Stats",
            "Fenerbahce": "/en/squads/ae1e2d7d/Fenerbahce-Stats",
            "Besiktas": "/en/squads/0f9294bd/Besiktas-Stats",
            "Trabzonspor": "/en/squads/c8ad3091/Trabzonspor-Stats",
            "Konyaspor": "/en/squads/5700c020/Konyaspor-Stats",
            "Goztepe": "/en/squads/7455853d/Goztepe-Stats",
            "Antalyaspor": "/en/squads/5ac76942/Antalyaspor-Stats",
            "Gaziantep_FK": "/en/squads/7c81865f/Gaziantep-FK-Stats",
            "Eyupspor": "/en/squads/e83a6881/Eyupspor-Stats",
            "Alanyaspor": "/en/squads/076128d7/Alanyaspor-Stats",
            "Fatih_Karagumruk": "/en/squads/bd61c555/Fatih-Karagumruk-Stats",
            "Basaksehir": "/en/squads/bff39cf5/Istanbul-Basaksehir-Stats",
            "Kayserispor": "/en/squads/1f33fbc7/Kayserispor-Stats",
            "Kocaelispor": "/en/squads/37fcbb73/Kocaelispor-Stats",
            "Rizespor": "/en/squads/f622e63c/Rizespor-Stats",
            "Kasimpasa": "/en/squads/d6ffefd1/Kasimpasa-Stats",
            "Genclerbirligi": "/en/squads/f6d47c93/Genclerbirligi-Stats",
            "Samsunspor": "/en/squads/77356e9e/Samsunspor-Stats"
        }
        
        team_urls = team_list_hardcoded # Doğrudan manuel listeyi kullan

        print(f"{len(team_urls)} takım URL'si bulundu.")

        # --- 2. Her Takımın Detay Sayfalarındaki Tabloları Çekme ---
        for team_name_safe, team_relative_url in team_urls.items():
            try:
                # `scrape_team_page_tables` fonksiyonunu çağırıyoruz
                scrape_team_page_tables(f"{base_url}{team_relative_url}", team_name_safe, output_base_directory, driver)
                time.sleep(5) # Her takım sayfası arasında daha uzun bekleme
            except Exception as e:
                print(f"Hata: '{team_name_safe}' takım sayfası işlenirken genel bir sorun oluştu: {e}")

    except Exception as e:
        print(f"Genel bir hata oluştu: {e}")
    finally:
        print("WebDriver kapatılıyor...")
        driver.quit()
        print("WebDriver kapatıldı.")

# --- Kodu çalıştırma bloğu (Jupyter Notebook için uygun) ---
if __name__ == "__main__":
    run_super_lig_scraping()

WebDriver başlatılıyor...
WebDriver başarıyla başlatıldı.
Süper Lig ana sayfasına gidiliyor: https://fbref.com/en/comps/26/Super-Lig-Stats
Süper Lig ana sayfası HTML içeriği çekildi ve ayrıştırıldı.
18 takım URL'si bulundu.

--- 'Galatasaray' takımının sayfasına gidiliyor: https://fbref.com/en/squads/ecd11ca2/Galatasaray-Stats ---
'Galatasaray' takımının HTML içeriği çekildi ve ayrıştırıldı.
Takım klasörü 'scraped_fbref_team_stats\Galatasaray' oluşturuldu.
  Takım için tablo okunuyor: 'Table_ID_unknown_table_0' (ID: unknown_table_0)...
  Hata: Takım sayfasındaki 'Table_ID_unknown_table_0' (ID: unknown_table_0) tablosu işlenirken sorun oluştu: 'int' object has no attribute 'strip'
  Takım için tablo okunuyor: 'Table_ID_unknown_table_1' (ID: unknown_table_1)...
  Hata: Takım sayfasındaki 'Table_ID_unknown_table_1' (ID: unknown_table_1) tablosu işlenirken sorun oluştu: 'int' object has no attribute 'strip'
  Takım için tablo okunuyor: 'Table_ID_unknown_table_2' (ID: unknown_table_2)...
  